In [6]:
import json
import os
import boto3
import rasterio
import pandas as pd

# Create metadata output file

In [17]:
out_dir = os.getcwd()
metadata_file = os.path.join(out_dir, 'metadata\\metadata_treecover_tml.json')

# Load input data

In [8]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv') 
boundary_georef = boundary_georef.dropna(subset=['units_boundary_name']).reset_index(drop=True)

In [9]:
# subset cities with avaialable tml data
tml_available_cities = ['Teresina city','Teresina region','Belem city','Belem region','San Jose','Kigali',
                        'Freetown city','Freetown region','Marrakech','Chennai region','Chennai region',
                        'Pune','Surat','Jakarta','Bitung','Semarang','Balikpapan','Balikpapan']
boundary_georef = boundary_georef.loc[boundary_georef['city_name'].isin(tml_available_cities)].reset_index(drop=True)
boundary_georef                        

,geo_name,level,aoi_boundary_name,units_boundary_name,city_name,country_name,country_code,continent
0,BRA-Teresina,city,ADM4union,ADM4,Teresina city,Brazil,BRA,America
1,BRA-Teresina,region,ADM2union,ADM2,Teresina region,Brazil,BRA,America
2,BRA-Belem,city,ADM4union,ADM4,Belem city,Brazil,BRA,America
3,BRA-Belem,region,ADM2union,ADM2,Belem region,Brazil,BRA,America
4,CRI-San_Jose,region,ADM2union,ADM2,San Jose,Costa Rica,CRI,America
5,RWA-Kigali,city,ADM4union,ADM4,Kigali,Rwanda,RWA,Africa
6,SLE-Freetown,city,ADM4cityunion,ADM4city,Freetown city,Sierra Leone,SLE,Africa
7,SLE-Freetown,region,ADM4regionunion,ADM4region,Freetown region,Sierra Leone,SLE,Africa
8,MAR-Marrakech,NaN,ADM2,ADM5,Marrakech,Morocco,MAR,Africa
9,IND-Chennai,region,ADM4union,ADM4,Chennai region,India,IND,Asia


# Create metadata file

In [18]:
dataset_metadata_cities = []

In [19]:
for i in range(0, len(boundary_georef)):
    
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    city_name = boundary_georef.loc[i, 'city_name']
    country_name = boundary_georef.loc[i, 'country_name']
    continent_name = boundary_georef.loc[i, 'continent']

    print(boundary_id)
    
    # read dataset
    dataset = rasterio.open('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/tree_cover/tree_mosaic_land/v_0/'+boundary_id+'-TML-tree_cover-2000.tif')

    # fill metadata fields
    dataset_title = "Trees in Mosaic Landscapes extract for " + city_name
    dataset_year = '2020'
    dataset_city_name = city_name
    dataset_country_name = country_name
    dataset_description = "The Trees in Mosaic Landscapes dataset provides tree extent data at 10m scale based on trained Convolutional Neural Network using satellite imagery (Sentinel-1 and Sentinel-2). It enables accurate reporting of tree cover outside of dense, closed-canopy forests, including in urban areas."
    dataset_snippet = "Tree cover data for "+ city_name + ', ' + country_name + ' ('+ dataset_year + ')' 
    dataset_tags = [geo_name,
                    "Greenspace",
                    "Biodiversity",
                    "Tree cover",
                    "Geography: " + continent_name,
                    "Geography: " + continent_name + ": " + country_name,
                    "Geography: " + continent_name + ": " + country_name + ": "+ city_name,
                    "Time: "+ dataset_year]
    dataset_spatial_resolution = '10m'
    dataset_temporal_resolution = 'yearly'
    dataset_spatial_extent = 'Global'
    dataset_temporal_extent = '2020'
    dataset_extent = [[dataset.bounds[0],dataset.bounds[1]],[dataset.bounds[2],dataset.bounds[3]]]
    dataset_format = 'raster'
    dataset_data_source = 'Trees in Mosaic Landscapes'
    dataset_source_url = 'https://www.landcarbonlab.org/data/#trees-in-mosaic-landscapes'
    dataset_provider = 'World Resources Institute (WRI)'
    dataset_url = "https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/tree_cover/tree_mosaic_land/v_0/"+boundary_id+"-TML-tree_cover-2000.tif"
    dataset_status = 'published'
    dataset_license = 'CC BY 4.0'
    dataset_crs = 'ESPG:4326'

    # create a dictionary
    dataset_metadata_city = {
        'title': dataset_title,
        'city_name': dataset_city_name,
        'country': dataset_country_name,
        'city_id': geo_name,
        'description': dataset_description,
        'snippet': dataset_snippet,
        'tags': dataset_tags,
        'year': dataset_year,
        'spatial resolution': dataset_spatial_resolution,
        'temporal resolution': dataset_temporal_resolution,
        'spatial extent': dataset_spatial_extent,
        'temporal extent': dataset_temporal_extent,
        'extent': dataset_extent,
        'format': dataset_format,
        'data_source': dataset_data_source,
        'source_url': dataset_source_url, 
        'provider': dataset_provider,
        'url': dataset_url,
        'status': dataset_status,
        'license': dataset_license,
        'crs': dataset_crs,
    }
    
    # append cities
    dataset_metadata_cities.append(dataset_metadata_city)
    

BRA-Teresina-ADM4union
BRA-Teresina-ADM2union
BRA-Belem-ADM4union
BRA-Belem-ADM2union
CRI-San_Jose-ADM2union
RWA-Kigali-ADM4union
SLE-Freetown-ADM4cityunion
SLE-Freetown-ADM4regionunion
MAR-Marrakech-ADM2
IND-Chennai-ADM4union
IND-Surat-ADM4union
IDN-Jakarta-ADM4union
IDN-Bitung-ADM2
IDN-Semarang-ADM1
IDN-Balikpapan-ADM4union


In [20]:
with open(metadata_file, 'w') as f:
    json.dump(dataset_metadata_cities, f)

In [15]:
# read dataset
dataset = rasterio.open('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/tree_cover/tree_mosaic_land/v_0/'+boundary_id+'-TML-tree_cover-2000.tif')


# Upload to aws s3

In [21]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-urbanshift' 

In [22]:
# upload in s3
s3.meta.client.upload_file(metadata_file, 
                           bucket_name, 
                           'data/tree_cover/tree_mosaic_land/v_0/metadata.json',
                           ExtraArgs={'ACL':'public-read'})